# Assignment 1: Search
# Part 2:

In [27]:
from search_1_2 import *
from itertools import combinations, permutations
import numpy as np
import time
    
fname = 'src/smallSearch.txt'
# sname_dfs = fname[:-4] + '_sol_dfs' + fname[-4:]
sname_bfs_1 = fname[:4] + 'p1_2_sol/' + fname[4:-4] + '_sol_bfs_1' + fname[-4:]
sname_bfs_2 = fname[:4] + 'p1_2_sol/' + fname[4:-4] + '_sol_bfs_2' + fname[-4:]
sname_gbfs = fname[:4] + 'p1_2_sol/' + fname[4:-4] + '_sol_gbfs' + fname[-4:]
sname_astar_1 = fname[:4] + 'p1_2_sol/' + fname[4:-4] + '_sol_astar_1' + fname[-4:]
sname_astar_2 = fname[:4] + 'p1_2_sol/' + fname[4:-4] + '_sol_astar_2' + fname[-4:]



# Breadth First Search (1)


In [28]:
maze_map, start, goal_dict, index_dict = parse_file(fname)

total_path = ''
total_expanded = 0


# print(goal_dict)
# print(index_dict)
timer_s = time.clock()

curr_node = start
while index_dict:
    dist = []
    for goal in index_dict:
        manhattan = get_manhattan(curr_node, goal)
        dist.append((manhattan, goal))
    if len(dist) == 1:
        print('DONE')
        break
    dist.sort()
#     print(dist)
    
    if len(dist) > 2:
        real_dist = []
        for i in range(1, len(dist) - 1):
            goal = dist[i][1]
            path, expanded = find_path_bfs(maze_map, start, goal)
            total_expanded += expanded
            bfs = len(path)
            real_dist.append((bfs, goal))
            if bfs > dist[i + 1][0]:
                break

        real_dist.sort()
#         print(real_dist)
        next_node = real_dist[0][1]
    else:
        next_node = dist[1][1]

    path, expanded = find_path_bfs(maze_map, curr_node, next_node)
    total_path += path
    
    print(curr_node, '--->>', next_node)
    curr_index = index_dict[curr_node]
    del index_dict[curr_node]
    del goal_dict[curr_index]
    curr_node = next_node

timer_t = time.clock()
print('bfs takes', timer_t - timer_s, 
      'seconds, costs', len(total_path), 
      'steps and expands', total_expanded, 'cells.')

maze_sol_bfs = draw_path(maze_map, total_path, start)
write_sol_to_file(sname_bfs_1, maze_sol_bfs)


(1, 7) --->> (1, 9)
(1, 9) --->> (4, 6)
(4, 6) --->> (4, 13)
(4, 13) --->> (4, 18)
(4, 18) --->> (6, 17)
(6, 17) --->> (9, 19)
(9, 19) --->> (11, 23)
(11, 23) --->> (7, 25)
(7, 25) --->> (1, 25)
(1, 25) --->> (1, 22)
(1, 22) --->> (3, 28)
(3, 28) --->> (1, 1)
(1, 1) --->> (6, 1)
(6, 1) --->> (10, 1)
(10, 1) --->> (11, 13)
DONE
bfs takes 0.012791758388289054 seconds, costs 180 steps and expands 1668 cells.


# Breadth First Search (2)


In [29]:
maze_map, start, goal_dict, index_dict = parse_file(fname)

total_path = ''
total_expanded = 0


# print(goal_dict)
print(index_dict)
timer_s = time.clock()

length = len(goal_dict.keys())

t = Tree(length)
dist_dict = {}
for goal_i, goal_j in combinations(index_dict.keys(), r = 2):
    path, expanded = find_path_bfs(maze_map, goal_i, goal_j)
    total_expanded += expanded
    dist = len(path)
    dist_dict[(goal_i, goal_j)] = dist
    dist_dict[(goal_j, goal_i)] = dist
    i = index_dict[goal_i]
    j = index_dict[goal_j]
    t.addEdge(i, j, dist)
    # print(i, j)
mst = t.KruskalMST()
print(mst)

curr_index = index_dict[start]
next_index = curr_index

while goal_dict.keys():
    length = len(goal_dict.keys())
    if length == 1:
        print('DONE')
        break
    
    edges = []
    for edge in mst:
        # print(curr_index)
        # print(edge)
        if edge[0] == curr_index or edge[1] == curr_index:
            edges.append(edge)
    for edge in edges:
        mst.remove(edge)
    
    if edges == []:
        dist_list = []
        curr_node = goal_dict[curr_index]
        for goal in index_dict.keys():
            if (curr_node, goal) in dist_dict.keys():
                dist_list.append((dist_dict[(curr_node, goal)], goal))
            else:
                path, expanded = find_path_bfs(maze_map, curr_node, goal)
                total_expanded += expanded
                dist = len(path)
                dist_list.append((dist, goal))
        dist_list.sort()
        if len(dist_list) == 1:
            print(index_dict)
            print(dist_list)
        next_node = dist_list[1][1]
    else:
        sorted(edges, key=lambda edge: edge[2])
        if edges[0][1] == curr_index:
            next_index = edges[0][0]
        else: 
            next_index = edges[0][1]
        curr_node = goal_dict[curr_index]
        next_node = goal_dict[next_index]
    
    print(curr_node, '--->>', next_node)
    path, expanded = find_path_astar(maze_map, curr_node, next_node)
    
    total_path += path
    
    del goal_dict[curr_index]
    del index_dict[curr_node]
    curr_node = next_node
    curr_index = index_dict[curr_node]

timer_t = time.clock()
print('bfs takes', timer_t - timer_s, 
      'seconds, costs', len(total_path), 
      'steps and expands', total_expanded, 'cells.')

maze_sol_bfs = draw_path(maze_map, total_path, start)
write_sol_to_file(sname_bfs_2, maze_sol_bfs)


{(1, 1): 0, (1, 7): 1, (1, 9): 2, (1, 22): 3, (1, 25): 4, (3, 28): 5, (4, 6): 6, (4, 13): 7, (4, 18): 8, (6, 1): 9, (6, 17): 10, (7, 25): 11, (9, 19): 12, (10, 1): 13, (11, 13): 14, (11, 23): 15}
[[1, 2, 2], [3, 4, 3], [4, 5, 5], [0, 1, 6], [7, 10, 6], [9, 13, 6], [11, 15, 6], [12, 15, 6], [2, 7, 7], [3, 8, 7], [6, 9, 7], [0, 6, 8], [12, 14, 10], [3, 15, 11], [7, 8, 11]]
(1, 7) --->> (1, 9)
(1, 9) --->> (4, 13)
(4, 13) --->> (6, 17)
(6, 17) --->> (4, 18)
(4, 18) --->> (1, 22)
(1, 22) --->> (1, 25)
(1, 25) --->> (3, 28)
(3, 28) --->> (11, 23)
(11, 23) --->> (7, 25)
(7, 25) --->> (9, 19)
(9, 19) --->> (11, 13)
(11, 13) --->> (4, 6)
(4, 6) --->> (6, 1)
(6, 1) --->> (10, 1)
(10, 1) --->> (1, 1)
DONE
bfs takes 0.06116497701827939 seconds, costs 130 steps and expands 12991 cells.


# Greedy Best First Search (1)

In [23]:
maze_map, start, goal_dict, index_dict = parse_file(fname)

total_path = ''
total_expanded = 0


# print(goal_dict)
print(index_dict)
timer_s = time.clock()

length = len(goal_dict.keys())

t = Tree(length)
dist_dict = {}
for goal_i, goal_j in combinations(index_dict.keys(), r = 2):
    path, expanded = find_path_gbfs(maze_map, goal_i, goal_j)
    total_expanded += expanded
    dist = len(path)
    dist_dict[(goal_i, goal_j)] = dist
    dist_dict[(goal_j, goal_i)] = dist
    i = index_dict[goal_i]
    j = index_dict[goal_j]
    t.addEdge(i, j, dist)
    # print(i, j)
mst = t.KruskalMST()
print(mst)

curr_index = index_dict[start]
next_index = curr_index

while goal_dict.keys():
    length = len(goal_dict.keys())
    if length == 1:
        print('DONE')
        break
    
    edges = []
    for edge in mst:
        # print(curr_index)
        # print(edge)
        if edge[0] == curr_index or edge[1] == curr_index:
            edges.append(edge)
    for edge in edges:
        mst.remove(edge)
    
    if edges == []:
        dist_list = []
        curr_node = goal_dict[curr_index]
        for goal in index_dict.keys():
            if (curr_node, goal) in dist_dict.keys():
                dist_list.append((dist_dict[(curr_node, goal)], goal))
            else:
                path, expanded = find_path_gbfs(maze_map, curr_node, goal)
                total_expanded += expanded
                dist = len(path)
                dist_list.append((dist, goal))
        dist_list.sort()
        if len(dist_list) == 1:
            print(index_dict)
            print(dist_list)
        next_node = dist_list[1][1]
    else:
        sorted(edges, key=lambda edge: edge[2])
        if edges[0][1] == curr_index:
            next_index = edges[0][0]
        else: 
            next_index = edges[0][1]
        curr_node = goal_dict[curr_index]
        next_node = goal_dict[next_index]
    
    print(curr_node, '--->>', next_node)
    path, expanded = find_path_astar(maze_map, curr_node, next_node)
    
    total_path += path
    
    del goal_dict[curr_index]
    del index_dict[curr_node]
    curr_node = next_node
    curr_index = index_dict[curr_node]

timer_t = time.clock()
print('gbfs takes', timer_t - timer_s, 
      'seconds, costs', len(total_path), 
      'steps and expands', total_expanded, 'cells.')

maze_sol_gbfs = draw_path(maze_map, total_path, start)
write_sol_to_file(sname_gbfs, maze_sol_gbfs)


{(1, 1): 0, (1, 7): 1, (1, 9): 2, (1, 22): 3, (1, 25): 4, (3, 28): 5, (4, 6): 6, (4, 13): 7, (4, 18): 8, (6, 1): 9, (6, 17): 10, (7, 25): 11, (9, 19): 12, (10, 1): 13, (11, 13): 14, (11, 23): 15}
[[1, 2, 2], [3, 4, 3], [4, 5, 5], [0, 1, 6], [7, 10, 6], [9, 13, 6], [11, 15, 6], [12, 15, 6], [2, 7, 7], [3, 8, 7], [6, 9, 7], [0, 6, 8], [3, 15, 11], [7, 8, 11], [7, 14, 13]]
(1, 7) --->> (1, 9)
(1, 9) --->> (4, 13)
(4, 13) --->> (6, 17)
(6, 17) --->> (4, 18)
(4, 18) --->> (1, 22)
(1, 22) --->> (1, 25)
(1, 25) --->> (3, 28)
(3, 28) --->> (11, 23)
(11, 23) --->> (7, 25)
(7, 25) --->> (9, 19)
(9, 19) --->> (11, 13)
(11, 13) --->> (4, 6)
(4, 6) --->> (6, 1)
(6, 1) --->> (10, 1)
(10, 1) --->> (1, 1)
DONE
gbfs takes 0.05307366574857042 seconds, costs 130 steps and expands 3388 cells.


# A* Search (1)

In [24]:
maze_map, start, goal_dict, index_dict = parse_file(fname)

total_path = ''
total_expanded = 0


# print(goal_dict)
# print(index_dict)
timer_s = time.clock()

curr_node = start
while index_dict:
    dist = []
    for goal in index_dict:
        manhattan = get_manhattan(curr_node, goal)
        dist.append((manhattan, goal))
    if len(dist) == 1:
        print('DONE')
        break
    dist.sort()
#     print(dist)
    
    if len(dist) > 2:
        real_dist = []
        for i in range(1, len(dist) - 1):
            goal = dist[i][1]
            path, expanded = find_path_astar(maze_map, start, goal)
            total_expanded += expanded
            bfs = len(path)
            real_dist.append((bfs, goal))
            if bfs > dist[i + 1][0]:
                break

        real_dist.sort()
#         print(real_dist)
        next_node = real_dist[0][1]
    else:
        next_node = dist[1][1]

    path, expanded = find_path_astar(maze_map, curr_node, next_node)
    total_path += path
    
    print(curr_node, '--->>', next_node)
    curr_index = index_dict[curr_node]
    del index_dict[curr_node]
    del goal_dict[curr_index]
    curr_node = next_node

timer_t = time.clock()
print('astar takes', timer_t - timer_s, 
      'seconds, costs', len(total_path), 
      'steps and expands', total_expanded, 'cells.')

maze_sol_astar = draw_path(maze_map, total_path, start)
write_sol_to_file(sname_astar_1, maze_sol_astar)


(1, 7) --->> (1, 9)
(1, 9) --->> (4, 6)
(4, 6) --->> (4, 13)
(4, 13) --->> (4, 18)
(4, 18) --->> (6, 17)
(6, 17) --->> (9, 19)
(9, 19) --->> (11, 23)
(11, 23) --->> (7, 25)
(7, 25) --->> (1, 25)
(1, 25) --->> (1, 22)
(1, 22) --->> (3, 28)
(3, 28) --->> (1, 1)
(1, 1) --->> (6, 1)
(6, 1) --->> (10, 1)
(10, 1) --->> (11, 13)
DONE
bfs takes 0.012146353576554247 seconds, costs 180 steps and expands 507 cells.


# A* Search (2)

In [25]:
maze_map, start, goal_dict, index_dict = parse_file(fname)

total_path = ''
total_expanded = 0


# print(goal_dict)
print(index_dict)
timer_s = time.clock()

length = len(goal_dict.keys())

t = Tree(length)
dist_dict = {}
for goal_i, goal_j in combinations(index_dict.keys(), r = 2):
    path, expanded = find_path_astar(maze_map, goal_i, goal_j)
    total_expanded += expanded
    dist = len(path)
    dist_dict[(goal_i, goal_j)] = dist
    dist_dict[(goal_j, goal_i)] = dist
    i = index_dict[goal_i]
    j = index_dict[goal_j]
    t.addEdge(i, j, dist)
    # print(i, j)
mst = t.KruskalMST()
print(mst)

curr_index = index_dict[start]
next_index = curr_index

while goal_dict.keys():
    length = len(goal_dict.keys())
    if length == 1:
        print('DONE')
        break
    
    edges = []
    for edge in mst:
        # print(curr_index)
        # print(edge)
        if edge[0] == curr_index or edge[1] == curr_index:
            edges.append(edge)
    for edge in edges:
        mst.remove(edge)
    
    if edges == []:
        dist_list = []
        curr_node = goal_dict[curr_index]
        for goal in index_dict.keys():
            if (curr_node, goal) in dist_dict.keys():
                dist_list.append((dist_dict[(curr_node, goal)], goal))
            else:
                path, expanded = find_path_astar(maze_map, curr_node, goal)
                total_expanded += expanded
                dist = len(path)
                dist_list.append((dist, goal))
        dist_list.sort()
        if len(dist_list) == 1:
            print(index_dict)
            print(dist_list)
        next_node = dist_list[1][1]
    else:
        sorted(edges, key=lambda edge: edge[2])
        if edges[0][1] == curr_index:
            next_index = edges[0][0]
        else: 
            next_index = edges[0][1]
        curr_node = goal_dict[curr_index]
        next_node = goal_dict[next_index]
    
    print(curr_node, '--->>', next_node)
    path, expanded = find_path_astar(maze_map, curr_node, next_node)
    
    total_path += path
    
    del goal_dict[curr_index]
    del index_dict[curr_node]
    curr_node = next_node
    curr_index = index_dict[curr_node]

timer_t = time.clock()
print('astar takes', timer_t - timer_s, 
      'seconds, costs', len(total_path), 
      'steps and expands', total_expanded, 'cells.')

maze_sol_astar = draw_path(maze_map, total_path, start)
write_sol_to_file(sname_astar_2, maze_sol_astar)


{(1, 1): 0, (1, 7): 1, (1, 9): 2, (1, 22): 3, (1, 25): 4, (3, 28): 5, (4, 6): 6, (4, 13): 7, (4, 18): 8, (6, 1): 9, (6, 17): 10, (7, 25): 11, (9, 19): 12, (10, 1): 13, (11, 13): 14, (11, 23): 15}
[[1, 2, 2], [3, 4, 3], [4, 5, 5], [0, 1, 6], [7, 10, 6], [9, 13, 6], [11, 15, 6], [12, 15, 6], [2, 7, 7], [3, 8, 7], [6, 9, 7], [0, 6, 8], [12, 14, 10], [3, 15, 11], [7, 8, 11]]
(1, 7) --->> (1, 9)
(1, 9) --->> (4, 13)
(4, 13) --->> (6, 17)
(6, 17) --->> (4, 18)
(4, 18) --->> (1, 22)
(1, 22) --->> (1, 25)
(1, 25) --->> (3, 28)
(3, 28) --->> (11, 23)
(11, 23) --->> (7, 25)
(7, 25) --->> (9, 19)
(9, 19) --->> (11, 13)
(11, 13) --->> (4, 6)
(4, 6) --->> (6, 1)
(6, 1) --->> (10, 1)
(10, 1) --->> (1, 1)
DONE
astar takes 0.05561119608103127 seconds, costs 130 steps and expands 4173 cells.
